### Extract SQL Table Information Schema in JSON Format

In [1]:
import sqlite3
import json

def get_table_info(database_path, table_name):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Query to get table information
    query = f"""
    SELECT
        name as column_name,
        type as data_type,
        [notnull] as is_nullable,
        dflt_value as column_default,
        pk as is_primary_key
    FROM
        pragma_table_info('{table_name}')
    ORDER BY
        cid;
    """

    cursor.execute(query)
    columns = [description[0] for description in cursor.description]
    rows = cursor.fetchall()

    # Convert the results to a list of dictionaries
    table_info = [dict(zip(columns, row)) for row in rows]

    # Close the connection
    conn.close()

    return table_info

def main():
    database_path = 'QTX_BankData.db'  # SQLite database path
    table_name = 'GL_Fact'  # Table name to inspect

    table_info = get_table_info(database_path, table_name)

    # Print the results as formatted JSON
    print(json.dumps(table_info, indent=2))

    # Optionally, you can also save the results to a file
    with open(f'{table_name}_info.json', 'w') as f:
        json.dump(table_info, f, indent=2)

if __name__ == "__main__":
    main()

[
  {
    "column_name": "Period",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "InterCo",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "GL Account",
    "data_type": "INTEGER",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "Project",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "Scenario",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "Entity",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "Measure",
    "data_type": "TEXT",
    "is_nullable": 0,
    "column_default": null,
    "is_primary_key": 0
  },
  {
    "column_name": "Pro

### Extract Summarized Loan Balances in JSON

In [1]:
import sqlite3
import json

def sum_loans_by_date(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    query = """
    SELECT 
        Period_EndDate,
        SUM(Value) as TotalValue,
        Scenario
    FROM 
        GL_Fact
    WHERE 
        [Major GL Account] = 'Assets'
    GROUP BY 
        Period_EndDate, Scenario
    ORDER BY 
        Period_EndDate, Scenario
    """

    try:
        cursor.execute(query)
        results = cursor.fetchall()

        # Convert results to a list of dictionaries
        json_data = [
            {
                "Period_EndDate": row[0],
                "Scenario": row[1],
                "TotalValue": row[2]
            }
            for row in results
        ]

        # Convert to JSON format
        json_output = json.dumps(json_data, indent=2)

        print("Query results:")
        print(json_output)

        # Optionally, save to a file
        with open('loans_sum_by_date.json', 'w') as f:
            f.write(json_output)
        print("\nResults have been saved to 'loans_sum_by_date.json'")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

def main():
    database_path = 'QTX_BankData.db'  # SQLite database path
    sum_loans_by_date(database_path)

if __name__ == "__main__":
    main()

Query results:
[
  {
    "Period_EndDate": "01/31/2022",
    "Scenario": 9680.0,
    "TotalValue": "Actual"
  },
  {
    "Period_EndDate": "01/31/2022",
    "Scenario": 2056.003599291299,
    "TotalValue": "Budget"
  },
  {
    "Period_EndDate": "01/31/2022",
    "Scenario": 10071.352510914894,
    "TotalValue": "Forecast"
  },
  {
    "Period_EndDate": "01/31/2023",
    "Scenario": 19132.0,
    "TotalValue": "Actual"
  },
  {
    "Period_EndDate": "01/31/2023",
    "Scenario": 3341.0,
    "TotalValue": "Budget"
  },
  {
    "Period_EndDate": "01/31/2023",
    "Scenario": 8421.0,
    "TotalValue": "Forecast"
  },
  {
    "Period_EndDate": "01/31/2024",
    "Scenario": 34812.24045709291,
    "TotalValue": "Actual"
  },
  {
    "Period_EndDate": "01/31/2024",
    "Scenario": 8857.0,
    "TotalValue": "Budget"
  },
  {
    "Period_EndDate": "01/31/2024",
    "Scenario": 3895.055312445194,
    "TotalValue": "Forecast"
  },
  {
    "Period_EndDate": "02/28/2022",
    "Scenario": 33012.38039